In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization
from model import yolov3
from IPython.core.display import display, HTML
from utils import *
from PIL import ImageDraw, Image

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "9"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
display(HTML("<style>.container { width:98% !important;}</style>"))

In [3]:
# load image
img = Image.open("/raid/peterchen/Project/Object_Detection_v2/predict/tim.jpg")
img_resized = img.resize(size=(416, 416))
img_resized = np.array(img_resized, dtype=np.float32)
img_resized = tf.expand_dims(tf.constant(img_resized)/255., axis=0)

In [4]:
# Build Model
yolov3_model = yolov3.model()
yolov3.load_weights("weights/yolov3.weights", yolov3_model)

In [5]:
model_output_1, model_output_2, model_output_3 = yolov3_model(img_resized)

In [6]:
print(model_output_1.shape)
print(model_output_2.shape)
print(model_output_3.shape)

(1, 13, 13, 255)
(1, 26, 26, 255)
(1, 52, 52, 255)


In [7]:
ANCHORS = [(10, 13), (16, 30), (33, 23), (30, 61), (62, 45), (59, 119), (116, 90), (159, 198), (373, 326)]

In [8]:
predictions_1 = post_process(model_output_1, (416, 416), ANCHORS[6:9], num_classes=80)
predictions_2 = post_process(model_output_2, (416, 416), ANCHORS[3:6], num_classes=80)
predictions_3 = post_process(model_output_3, (416, 416), ANCHORS[0:3], num_classes=80)

In [9]:
print(predictions_1.shape)
print(predictions_2.shape)
print(predictions_3.shape)

(1, 507, 85)
(1, 2028, 85)
(1, 8112, 85)


In [10]:
detections = tf.concat([predictions_1, predictions_2, predictions_3], axis=1)

In [11]:
boxes = center_to_minmax(detections)

In [12]:
conf_threshold = 0.5
iou_threshold = 0.4
class_names = "coco.names"

In [13]:
filtered_boxes = non_max_suppression(boxes, confidence_threshold=conf_threshold, iou_threshold=iou_threshold)

shape1 (10647, 85)
shape2 (13, 85)


In [14]:
classes = load_coco_names(class_names)

In [15]:
draw_boxes(filtered_boxes, img, classes, (416, 416))

person
 99.98% [270.6526301457332, 35.95089252178485]
person
 99.94% [423.85929987980774, 52.04748300405649]
sofa
 64.87% [578.8946063701924, 216.68053260216345]
chair
 95.51% [101.7606177696815, 266.3038400503305]
chair
 84.06% [423.7697190504808, 259.4697805551382]
chair
 84.06% [342.87135197566107, 305.1655813363882]
remote
 51.65% [327.5783362755409, 155.38773756760816]


In [16]:
img.save("aaa.jpg")